In [44]:
import numpy as np
import pandas as pd
from scipy.stats import dirichlet, expon, uniform, norm
from faker import Faker
fake = Faker()

In [45]:
# Define some global parameters
N_DAYS = 365*2
N_AUTHORS = 50

# Define possible topics and baseline probabilities // right now all made up, but should be based on data eventually
topics = ['Opinion', 'Politics', 'World Events', 'Business', 'Technology', 'Arts & Culture', 'Sports', 'Health', 'Home', 'Travel', 'Fashion', 'Food']
topics_probs = np.array([[0.1, 0.1, 0.1, 0.1, 0.08, 0.08, 0.08, 0.08, 0.08, 0.07, 0.07, 0.06]])

# Generate fake authors and probability vectors
authors_names = [fake.name() for i in range(N_AUTHORS)]

alphas = np.ones(len(topics)) # Concentration parameters TBD, right now uniform
authors_topicsprobs = np.array([dirichlet.rvs(alphas)[0] for auth in authors_names])

events = []
articles = []

# Iterate over days to generate all articles
for day in range(N_DAYS):

    # Generate new event with some probability (TBD)
    for _ in range(0, int(norm.rvs(loc=1, scale=2))):
        event_influence = dirichlet.rvs(alphas) # Concentration parameters TBD
        event_duration = expon.rvs(loc=0.01, scale=0.1) # some events should be long-lived
        duration_days = np.ceil(event_duration * 30)
        # hmm, event duration and intensity is generally correlated
        event_intensity = expon.rvs(scale=0.1, loc=0.1)
        events.append({'id': len(events), 'start': day, 'influence': event_influence, 'duration': duration_days, 'intensity': event_intensity})

    # Loop through events and sum topic probabilities of all active events
    day_topicsprobs = topics_probs.copy()
    for event in events:
        if event['start'] + event['duration'] >= day:
            day_topicsprobs += event['intensity']*event['influence']

    # Normalize probabilities
    day_topicsprobs = day_topicsprobs[0] / day_topicsprobs[0].sum()

    # Generate articles for day
    n_articles = round(norm.rvs(loc=N_AUTHORS / 3, scale=N_AUTHORS / 10))
    for i in range(n_articles):
        article_topic = np.random.choice(topics, p=day_topicsprobs)
        # Normalize probabilities
        author_probs = authors_topicsprobs[:, topics.index(article_topic)]
        author_probs = author_probs / author_probs.sum()
        article_author = np.random.choice(authors_names, p=author_probs)
        articles.append({'id': len(articles), 'day': day, 'topic': article_topic, 'author': article_author})

events_df = pd.DataFrame(events)
articles_df = pd.DataFrame(articles)

events_df.describe()

,id,start,duration,intensity
count,815.000000,815.000000,815.000000,815.000000
mean,407.000000,370.957055,3.777914,0.201271
std,235.414528,202.680112,3.035580,0.101339
min,0.000000,2.000000,1.000000,0.100012
25%,203.500000,203.500000,2.000000,0.128322
50%,407.000000,368.000000,3.000000,0.170034
75%,610.500000,550.000000,5.000000,0.245088
max,814.000000,728.000000,20.000000,0.828157


In [46]:
articles_df.describe()

,id,day
count,12211.000000,12211.000000
mean,6105.000000,362.827205
std,3525.156403,209.643456
min,0.000000,0.000000
25%,3052.500000,181.000000
50%,6105.000000,361.000000
75%,9157.500000,542.000000
max,12210.000000,729.000000


In [47]:
articles_df.groupby('topic').describe()

id                                                   \
                 count         mean          std   min      25%     50%   
topic                                                                     
Arts & Culture   987.0  6289.624113  3577.151613   6.0  3072.50  6380.0   
Business        1119.0  6116.484361  3482.022229   9.0  3076.50  6126.0   
Fashion         1023.0  6412.439883  3473.023459   4.0  3565.50  6557.0   
Food             850.0  6107.778824  3471.910774   1.0  3222.50  6071.5   
Health           951.0  6034.329127  3522.441204   8.0  3011.00  6090.0   
Home             978.0  6027.558282  3617.203052   0.0  2762.25  6089.5   
Opinion         1165.0  6022.834335  3567.824582  35.0  2928.00  5874.0   
Politics        1108.0  5911.452166  3522.025141   3.0  2844.25  5727.0   
Sports          1026.0  6195.890838  3440.064698  39.0  3288.00  6316.5   
Technology      1012.0  5921.337945  3535.292790  47.0  2790.75  5877.5   
Travel           929.0  5967.893434  3549.144762   2.0  2810.00  5712.0   
World Events    1063.0  6256.604892  3518.519978  26.0  3326.00  6304.0   

                                     day                                       \
                    75%      max   count        mean         std  min     25%   
topic                                                                           
Arts & Culture  9461.50  12168.0   987.0  373.939210  212.874279  0.0  182.50   
Business        9090.50  12187.0  1119.0  363.475424  207.033444  0.0  182.50   
Fashion         9248.50  12207.0  1023.0  381.137830  206.712965  0.0  212.00   
Food            9056.50  12208.0   850.0  362.980000  206.552066  0.0  189.50   
Health          9061.50  12195.0   951.0  358.599369  209.434689  0.0  179.00   
Home            9328.50  12194.0   978.0  358.329243  215.127112  0.0  165.00   
Opinion         9236.00  12210.0  1165.0  358.091845  212.278150  1.0  175.00   
Politics        8999.25  12206.0  1108.0  351.195848  209.288636  0.0  169.75   
Sports          8994.00  12180.0  1026.0  368.159844  204.558842  2.0  193.25   
Technology      8977.25  12201.0  1012.0  351.848814  210.015618  2.0  166.00   
Travel          9234.00  12205.0   929.0  354.617869  211.004569  0.0  167.00   
World Events    9302.00  12191.0  1063.0  371.796802  209.334899  1.0  195.50   

                                      
                  50%     75%    max  
topic                                 
Arts & Culture  381.0  561.00  727.0  
Business        362.0  537.00  728.0  
Fashion         391.0  547.50  729.0  
Food            358.0  535.00  729.0  
Health          360.0  535.00  729.0  
Home            360.0  552.75  729.0  
Opinion         347.0  547.00  729.0  
Politics        338.0  532.25  729.0  
Sports          375.5  532.00  728.0  
Technology      347.0  531.50  729.0  
Travel          337.0  547.00  729.0  
World Events    375.0  552.00  728.0